In [1]:
import os
import sys
import time
import base64
import uuid
import pandas as pd
from zipfile import ZipFile
from lxml import etree
import xml.etree.ElementTree as ET
import codecs
import json
from itertools import groupby

utilities_dir = '/Users/kd/Workspace/python/helpers'
sys.path.append(utilities_dir)

from file_directory_utils import (create_directory, read_directory_files, get_subdirectories, get_all_file_paths)


In [2]:
input_filepath = '/Users/kd/Workspace/python/DOCX/document-formatting/data/input/demo_judgment_1.docx'
output_dir     = '/Users/kd/Workspace/python/DOCX/document-formatting/data/output'

fetch_content_filepath = '/Users/kd/Workspace/python/DOCX/document-formatting/data/input/demo_judgment_1.json'
filename       = os.path.splitext(os.path.basename(input_filepath))[0]


In [3]:
def get_string_xmltree(xml):
    return etree.tostring(xml)

def get_xml_tree(xml_string):
    return etree.fromstring(xml_string)

def get_xmltree(filepath, parse='xml'):
    if parse == 'html':
        parser = etree.HTMLParser()
        tree   = etree.parse(open(filepath, mode='r', encoding='utf-8'), parser)
        return tree
    else:
        with open(filepath,'r') as file:
            xml_string    = file.read()
            return etree.fromstring(bytes(xml_string, encoding='utf-8'))
    return None

def check_element_is(element, type_char):
    word_schema1 = "http://schemas.openxmlformats.org/wordprocessingml/2006/main"
    word_schema2 = 'http://purl.oclc.org/ooxml/wordprocessingml/main'
     
    return (element.tag == '{%s}%s' % (word_schema1, type_char)) or (element.tag == '{%s}%s' % (word_schema2, type_char))

def get_specific_tags(node, type_char):
    nodes = []
    for elem in node.iter():
        if check_element_is(elem, type_char):
            nodes.append(elem)
    return nodes

def add_identifier(node):
    node.attrib['id'] = str(uuid.uuid4())

def is_run_superscript(run):
    attrib    = {}
    vertAlign = get_specific_tags(run, 'vertAlign')
    if len(vertAlign) > 0:
        for key in vertAlign[0].attrib.keys():
            attrib['vertAlign_' + key.split('}')[-1]] = vertAlign[0].attrib[key]
    if 'vertAlign_val' in attrib:
        if attrib['vertAlign_val'] == 'superscript':
            return True
    return False
    
def update_run_text(r1, r2):
    t1s = get_specific_tags(r1, 't')
    t2s = get_specific_tags(r2, 't')
#     print('r1 text [%s], r2 text [%s]'% (t1s[0].text, t2s[0].text))
    t1s[0].text = t1s[0].text + t2s[0].text
    t2s[0].text = ''
    
def get_run_properties(run):
    attrib = {}
    rFonts = get_specific_tags(run, 'rFonts')
    sz     = get_specific_tags(run, 'sz')
    szCs   = get_specific_tags(run, 'szCs')
    
    if len(rFonts) > 0:
        for key in rFonts[0].attrib.keys():
            attrib['rFonts_' + key.split('}')[-1]] = rFonts[0].attrib[key]
    
    if len(sz) > 0:
        for key in sz[0].attrib.keys():
            attrib['sz_' + key.split('}')[-1]] = sz[0].attrib[key]
        
    if len(szCs) > 0:
        for key in szCs[0].attrib.keys():
            attrib['szCs_' + key.split('}')[-1]] = szCs[0].attrib[key]

    return attrib

def compare_run_properties(run1, run2):
    attrib1 = get_run_properties(run1)
    attrib2 = get_run_properties(run2)
    
    if all (k in attrib1 for k in ('rFonts_ascii', 'sz_val', 'szCs_val')):
        if all (k in attrib2 for k in ('rFonts_ascii', 'sz_val', 'szCs_val')):
            if (attrib1['rFonts_ascii'] == attrib2['rFonts_ascii']) and \
            (attrib1['szCs_val'] == attrib2['szCs_val']) and \
            (attrib1['sz_val'] == attrib2['sz_val']) :
                return True
    
    return False

def get_line_connections(p):
    runs             = get_specific_tags(p, 'r')
    text_runs        = []
    
    for run in runs:
        if is_run_superscript(run) == False:
            text_runs.append(run)

    line_connections = []
    for index in range(len(text_runs) - 1):
        if (compare_run_properties(text_runs[index], text_runs[index+1])):
            line_connections.append((index, index+1, 'CONNECTED'))
        else:
            line_connections.append((index, index+1, 'NOT_CONNECTED'))
    return line_connections

def arrange_grouped_line_indices(line_connections, debug=False):
    lines          = [list(i) for j, i in groupby(line_connections, lambda a: a[2])]
    if debug:
        print('arrange_grouped_line_indices: %s \n---------\n' % (str(lines)))
        
    arranged_lines = []

    for line_items in lines:
        indices = []
        for line_item in line_items:
            indices.append(line_item[0])
            indices.append(line_item[1])
        indices = sorted(list(set(indices)))
        arranged_lines.append([indices, line_items[0][2]])
        
    if debug:
        print('arrange_grouped_line_indices,arranged_lines : %s \n---------\n' % (str(arranged_lines)))
    
    final_arranged_lines = []
    
    if len(arranged_lines) == 1:
        final_arranged_lines.append([arranged_lines[0][0], arranged_lines[0][1]])
    else:
        for index, line_item in enumerate(arranged_lines):
            if index == 0 and line_item[1] == 'NOT_CONNECTED':
                del line_item[0][-1]
            if index > 0 and index < (len(arranged_lines) - 1) and line_item[1] == 'NOT_CONNECTED':
                del line_item[0][0]
                del line_item[0][-1]
            if index == (len(arranged_lines) - 1) and line_item[1] == 'NOT_CONNECTED':
                del line_item[0][0]

            final_arranged_lines.append([line_item[0], line_item[1]])
    if debug:
        print('final_arrange_grouped_line_indices,arranged_lines : %s \n---------\n' % (str(final_arranged_lines)))
            
    return final_arranged_lines

def merge_runs(node, grouped_runs, debug=False):
    runs      = get_specific_tags(node, 'r')
    text_runs = []
    
    for run in runs:
        if is_run_superscript(run) == False:
            text_runs.append(run)

    for element in grouped_runs:
        if (element[1] == 'CONNECTED'):
            for index, run_index in enumerate(element[0]):
                if (index > 0):
                    if (debug):
                        print('merge index %d with %d' % ( run_index, 0))
                    update_run_text(text_runs[0], text_runs[run_index])
                    
def update_document_runs(document):
    '''
    the function iterates through the p tags and merges run that have exactly same
    visual property.
    '''
    tag_name                 = 'p'
    tags                     = get_specific_tags(document, tag_name)
    for p in tags:
        grouped_runs = arrange_grouped_line_indices(get_line_connections(p))
        merge_runs(p, grouped_runs, debug=False)
    return document

def get_text_tags(document):
    tags         = []
    texts        = get_specific_tags(document, 't')
    for text in texts:
        if text.text and len(text.text.strip()) > 0:
            add_identifier(text)
            tags.append(text)
    return tags

In [4]:
def extract_docx(filepath, working_dir):
    filename       = os.path.splitext(os.path.basename(filepath))[0]
    extract_dir    = os.path.join(working_dir, filename)
    
    with ZipFile(filepath, 'r') as file:
        file.extractall(path=extract_dir)
        filenames = file.namelist()
    
    return extract_dir, filenames

def save_docx(extracted_dir, filenames, output_filename):
    with ZipFile(output_filename, 'w') as docx:
        for filename in filenames: 
            docx.write(os.path.join(extracted_dir, filename), filename)
            
def save_document_xml(extracted_dir, xml):
    with open(os.path.join(extracted_dir,'word/document.xml'), 'wb') as f:
        xmlstr = get_string_xmltree(xml)
        f.write(xmlstr)

In [5]:
def get_tokenized_sentences(filepath):
    from jsonpath_rw import jsonpath, parse
    json_data     = json.load(codecs.open(fetch_content_filepath, 'r', 'utf-8-sig'))
    jsonpath_expr = parse('$..tokenized_sentences[*]')
    matches       = jsonpath_expr.find(json_data)

    tokenized_sentences = []
    for match in matches:
        tokenized_sentences.append(match.value)
    
    return tokenized_sentences

In [6]:
def count_occurrences(string, substring): 
    count = 0
    start = 0
    while start < len(string): 
        pos = string.find(substring, start) 

        if pos != -1: 
            start = pos + 1
            count += 1
        else: 
            break
    return count 

def check_string_status(doc_tag, tokenized):
    doc_text       = doc_tag.text.replace(" ", "")
    tokenized_text = tokenized['src'].replace(" ", "")
    
    if len(doc_text) < 2 or len(tokenized_text) < 2:
        if doc_text.isdigit() == False or tokenized_text.isdigit() == False:
            return (False, False)
    
    '''
        perfect match
    '''
    if doc_text == tokenized_text:
        return (True, 0)
    
    count         = 0
    if len(doc_text) > len(tokenized_text):
        count = count_occurrences(doc_text, tokenized_text)
        if count != 0:
            return (True, -1)
    else:
        count = count_occurrences(tokenized_text, doc_text)
        if count != 0:
            return (True, 1)
    
    return (False, False)



In [7]:
def get_as_df(tags, tokenized_sentences):
    doc_texts = []
    doc_ids   = []
    for tag in tags:
        doc_texts.append(tag.text)
        doc_ids.append(tag.attrib['id'])

    tokenized_src_texts = []
    tokenized_tgt_texts = []
    for tokenized_sentence in tokenized_sentences:
        tokenized_src_texts.append(tokenized_sentence['src'])
        tokenized_tgt_texts.append(tokenized_sentence['tgt'])

    if len(doc_texts) > len(tokenized_src_texts):
        empty = [''] * (len(doc_texts) - len(tokenized_src_texts))
        tokenized_src_texts.extend(empty)
        tokenized_tgt_texts.extend(empty)
    else:
        empty = [''] * (len(tokenized_src_texts) - len(doc_texts))
        doc_texts.extend(empty)
        doc_ids.extend(empty)

    df = pd.DataFrame(list(zip(doc_texts, doc_ids, tokenized_src_texts, tokenized_tgt_texts)), 
                                  columns =['doc_texts', 'doc_ids', 'tokenized_src_texts', 'tokenized_tgt_texts'])
    return df

In [8]:
def replace_translated(placeholders, texts):
    for placeholder in placeholders:
        for text in texts:
            if 'id' in text.attrib:
                if text.attrib['id'] == placeholder['id']:
                    text.text = placeholder['tgt']

In [9]:
extracted_dir, filenames = extract_docx(input_filepath, output_dir)


In [10]:
document_xml             = get_xmltree(os.path.join(extracted_dir, 'word', 'document.xml'))
document_xml             = update_document_runs(document_xml)
texts                    = get_text_tags(document_xml)

tokenized_sentences      = get_tokenized_sentences(fetch_content_filepath)
print('document has (%d) text tags, tokenized sentences (%d)' % (len(texts), len(tokenized_sentences)))


document has (54) text tags, tokenized sentences (46)


In [12]:
df = get_as_df(texts, tokenized_sentences)
df.head(60)


,doc_texts,doc_ids,tokenized_src_texts,tokenized_tgt_texts
0,Reportable,b9094355-2157-4399-8977-bdf6a2c00cf3,Reportable,रिपोर्ट योग्य
1,IN THE SUPREME COURT OF INDIA,cdf87c88-9b77-4c2d-82cc-cb570d1c2f1b,IN THE SUPREME COURT OF INDIA,भारत के सर्वोच्च न्यायालय में
2,CIVIL APPELLATE JURISDICTION,011e55e0-09a9-4373-97a3-593b864d287b,CIVIL APPELLATE JURISDICTION,दीवानी अपीलीय क्षेत्राधिकार
3,Civil Appeal Nos 9367-9369 of 2011,0c44e697-6384-49dc-ae01-b090c8dac730,Civil Appeal Nos 9367-9369 of 2011,9367 की दीवानी अपील सं. 2011-9369
4,"The Secretary, Ministry of Defence",d1cc1bb9-2050-4f10-a3bc-3167fd590d01,"The Secretary, Ministry of Defence",रक्षा मंत्रालय के सचिव
5,...Appellant,034c6aaf-6150-4578-8b89-8adcd1c0bfc3,......Appellant,...... अपीलार्थी
6,Versus,fc244334-ac68-4743-aa48-733454d14a0f,Versus,बनाम
7,Babita Puniya & Ors.,d0a6c4e6-b373-4704-b317-c0f97eb73c8d,Babita Puniya & Ors.,बबिता पुनिया और ओआरएस।
8,...Respondents,e278a839-b7d4-451e-921e-83f79007d425,......Respondents,..... उत्तरदाता
9,With,13755d7d-8f1f-4034-83e4-d159cfe01e3e,With,साथ में


In [13]:
## re-attempt
matched_ids    = []
matched_sids   = []

perfect_match   = []
tokenized_match = []
text_match      = []


for sent_index in range(len(tokenized_sentences)):
    for text_index in range(len(texts)):
        if (texts[text_index].attrib['id'] not in matched_ids) and \
        (tokenized_sentences[sent_index]['s_id'] not in matched_sids):
            is_found, is_substring = check_string_status(texts[text_index], tokenized_sentences[sent_index])
            
            if is_found and is_substring == 0:
                matched_ids.append(texts[text_index].attrib['id'])
                matched_sids.append(tokenized_sentences[sent_index]['s_id'])
                perfect_match.append({
                    's_id': tokenized_sentences[sent_index]['s_id'],
                    'src': tokenized_sentences[sent_index]['src'],
                    'tgt': tokenized_sentences[sent_index]['tgt'],
                    'id': texts[text_index].attrib['id'],
                    'text': texts[text_index].text
                })
                
            if is_found and is_substring < 0:
                tokenized_match.append({
                    's_id': tokenized_sentences[sent_index]['s_id'],
                    'src': tokenized_sentences[sent_index]['src'],
                    'tgt': tokenized_sentences[sent_index]['tgt'],
                    'id': texts[text_index].attrib['id'],
                    'text': texts[text_index].text
                })
                
            if is_found and is_substring > 0:
                text_match.append({
                    's_id': tokenized_sentences[sent_index]['s_id'],
                    'src': tokenized_sentences[sent_index]['src'],
                    'tgt': tokenized_sentences[sent_index]['tgt'],
                    'id': texts[text_index].attrib['id'],
                    'text': texts[text_index].text
                })

print('perfect match %d, tokenized match %d, text_match %d' % (len(perfect_match), len(tokenized_match), len(text_match)))
print('tokenized sentences count %d, texts count %d' % (len(tokenized_sentences), len(texts)))

perfect match 20, tokenized match 3, text_match 28
tokenized sentences count 46, texts count 54


In [16]:
text_match

[{'id': '034c6aaf-6150-4578-8b89-8adcd1c0bfc3',
  's_id': 'd0269ae7-a816-4018-bc29-7e5068e2ab44',
  'src': '......Appellant',
  'text': '...Appellant',
  'tgt': '...... अपीलार्थी'},
 {'id': 'e278a839-b7d4-451e-921e-83f79007d425',
  's_id': '4e372407-acc7-4774-a836-f02446ef9ed3',
  'src': '......Respondents',
  'text': '...Respondents',
  'tgt': '..... उत्तरदाता'},
 {'id': 'c8b04270-9e11-4f8a-a344-ea30f7614a70',
  's_id': 'aca36b8a-674e-4732-80c3-05b050e5ce25',
  'src': 'Signature Not Verified Digitally signed by',
  'text': 'Signature Not Verified',
  'tgt': 'हस्ताक्षर निर्दिष्ट डिजिटल रूप से हस्ताक्षरित नहीं'},
 {'id': '3d734258-4da1-4672-b9bf-eb958f0b5996',
  's_id': 'aca36b8a-674e-4732-80c3-05b050e5ce25',
  'src': 'Signature Not Verified Digitally signed by',
  'text': 'Digitally signed by',
  'tgt': 'हस्ताक्षर निर्दिष्ट डिजिटल रूप से हस्ताक्षरित नहीं'},
 {'id': 'f86be697-4414-4bde-95c6-a5c0295e8b0b',
  's_id': '1569aa27-d86e-4cd2-b695-3232cc7af476',
  'src': '13:49:23 IST Reason:',

In [15]:
perfect_match

[{'id': 'b9094355-2157-4399-8977-bdf6a2c00cf3',
  's_id': 'c313cd23-6764-483e-bc1b-fff0525b1e2d',
  'src': 'Reportable',
  'text': 'Reportable',
  'tgt': 'रिपोर्ट योग्य'},
 {'id': 'cdf87c88-9b77-4c2d-82cc-cb570d1c2f1b',
  's_id': 'a1150d63-a0ff-417d-a4a9-616ba45ceb9b',
  'src': 'IN THE SUPREME COURT OF INDIA',
  'text': 'IN THE SUPREME COURT OF INDIA',
  'tgt': 'भारत के सर्वोच्च न्यायालय में'},
 {'id': '011e55e0-09a9-4373-97a3-593b864d287b',
  's_id': '582dd1a5-90c3-4844-8b7d-e9374e46b2fa',
  'src': 'CIVIL APPELLATE JURISDICTION',
  'text': 'CIVIL APPELLATE JURISDICTION',
  'tgt': 'दीवानी अपीलीय क्षेत्राधिकार'},
 {'id': '0c44e697-6384-49dc-ae01-b090c8dac730',
  's_id': '996d557b-658b-4803-a61b-08fdfaca92fd',
  'src': 'Civil Appeal Nos 9367-9369 of 2011',
  'text': 'Civil Appeal Nos 9367-9369 of 2011',
  'tgt': '9367 की दीवानी अपील सं. 2011-9369'},
 {'id': 'd1cc1bb9-2050-4f10-a3bc-3167fd590d01',
  's_id': '60d82391-6c0d-4912-8a14-fd181600c817',
  'src': 'The Secretary, Ministry of Defen

In [14]:
text_match_dict = {}
for elem in text_match:
    if elem['s_id'] not in text_match_dict.keys():
        text_match_dict[elem['s_id']] = [elem]
    else:
        text_match_dict[elem['s_id']].append(elem)

In [15]:
unique_text_match = []
multiple_text_match_dict = {}

for key in text_match_dict.keys():
    if len(text_match_dict[key]) > 1:
        multiple_text_match_dict[key] = text_match_dict[key]
    else:
        unique_text_match.append(text_match_dict[key][0])


In [16]:
for key in multiple_text_match_dict.keys():
    doc_text = ''
    for elem in multiple_text_match_dict[key]:
        doc_text = doc_text + ' ' + elem['text']
        
    src_list = [x for x in elem['src'].split(' ') if x]
    src_set  = set(src_list)
    
    doc_list = [x for x in doc_text.split(' ') if x]
    doc_set  = set(doc_list)
    if len(doc_list) <= len(src_list):
        if (len(doc_set) <= len(src_set.intersection(doc_set))) and \
        len(src_set.difference(doc_set)) <= 2:
            unique_text_match.append(multiple_text_match_dict[key][0])
    else:
        print('repeating text')
        print(doc_list, '\n')
        print(src_list, '\n')

repeating text
['Background', 'of', 'the', 'dispute', 'Background', 'of', 'the', 'dispute'] 

['A', 'Background', 'of', 'the', 'dispute'] 

repeating text
['Background', 'of', 'the', 'dispute', 'Background', 'of', 'the', 'dispute'] 

['A', 'Background', 'of', 'the', 'dispute'] 



In [17]:
replace_translated(perfect_match, texts)
replace_translated(unique_text_match, texts)

replace_translated(tokenized_match, texts)


In [18]:
save_document_xml(extracted_dir, document_xml)
save_docx(extracted_dir, filenames, os.path.join(output_dir, "kd2.docx"))

In [11]:
tokenized_sentences = tokenized_sentences
tags                = texts
texts_matched       = []
placeholders        = []

for sent_index in range(len(tokenized_sentences)):
    for tag_index in range(len(tags)):
        if (tags[tag_index].attrib['id'] not in texts_matched) and \
            (tokenized_sentences[sent_index]['s_id'] not in texts_matched):
            found, tag_text_substring = check_string_status(tags[tag_index], tokenized_sentences[sent_index])
            if found == True:
#                 print('%s <-> %s <--> %r' % (tags[tag_index].text, tokenized_sentences[sent_index]['src'], tag_text_substring))
                if tag_text_substring == None:
                    texts_matched.append(tags[tag_index].attrib['id'])
                    texts_matched.append(tokenized_sentences[sent_index]['s_id'])
                    placeholders.append({
                        's_id': tokenized_sentences[sent_index]['s_id'],
                        'src': tokenized_sentences[sent_index]['src'],
                        'tgt': tokenized_sentences[sent_index]['tgt'],
                        'id': tags[tag_index].attrib['id'],
                        'text': tags[tag_index].text
                    })
                if tag_text_substring == True:
                    texts_matched.append(tokenized_sentences[sent_index]['s_id'])
                    placeholders.append({
                        's_id': tokenized_sentences[sent_index]['s_id'],
                        'src': tokenized_sentences[sent_index]['src'],
                        'tgt': tokenized_sentences[sent_index]['tgt'],
                        'id': tags[tag_index].attrib['id'],
                        'text': tags[tag_index].text
                    })
        else:
            pass
#             print('tag_id %s %s already matched' % (tags[tag_index].attrib['id'], tags[tag_index].text))

'''
    check how much tokenized didn't match
'''
unmatched_tokenized_sentences = []
for tokenized_sentence in tokenized_sentences:
    if tokenized_sentence['s_id'] in texts_matched:
        pass
    else:
        print('unmatched tokenized text:-> %s' % (tokenized_sentence['src']))
        unmatched_tokenized_sentences.append(tokenized_sentence)

print('tokenized sentences, initial: %d, matched: %d, unmatched: %d' % (len(tokenized_sentences), len(placeholders), len(unmatched_tokenized_sentences)))

unmatched_tags = []
for tag in tags:
    if tag.attrib['id'] in texts_matched:
        pass
    else:
        print('unmatched tag text:-> %s' % (tag.text))
        unmatched_tags.append(tag.text)


unmatched tokenized text:-> G Blanket restriction on criteria appointments
tokenized sentences, initial: 48, matched: 47, unmatched: 1
unmatched tag text:-> ...Appellant
unmatched tag text:-> 
unmatched tag text:-> 
unmatched tag text:-> 
unmatched tag text:-> 
unmatched tag text:-> 
unmatched tag text:-> ...Respondents
unmatched tag text:-> Signature Not Verified
unmatched tag text:-> SANJAY KUMAR
unmatched tag text:-> 13:49:23 IST
unmatched tag text:-> Reason:
unmatched tag text:-> 
unmatched tag text:-> 
unmatched tag text:-> 
unmatched tag text:-> 
unmatched tag text:-> Background of the dispute
unmatched tag text:-> Proposal of the Union of India
unmatched tag text:-> Submissions
unmatched tag text:-> Consequence of the policy letter dated 25 February 2019
unmatched tag text:-> Stereotypes and women in the Armed Forces
unmatched tag text:-> Consequence of non-compliance
unmatched tag text:-> Directions
unmatched tag text:-> A
unmatched tag text:-> Background of the dispute
unmatch

In [12]:
placeholders

[{'id': '979e0e55-6333-447a-8976-ed8cb8ec82be',
  's_id': '78844881-23a5-4a12-955e-ddc7b2df8c1d',
  'src': 'Reportable',
  'text': 'Reportable',
  'tgt': 'रिपोर्ट योग्य'},
 {'id': '3da4ac2a-b07b-4304-80d0-6675c1fa050f',
  's_id': '0a8ee6fb-bc27-482e-ab47-7169c2acccbb',
  'src': 'IN THE SUPREME COURT OF INDIA',
  'text': 'IN THE SUPREME COURT OF INDIA',
  'tgt': 'भारत के सर्वोच्च न्यायालय में'},
 {'id': '7758957c-f62a-46bf-9c58-4b1b992bfd38',
  's_id': '65b11398-df84-4199-aad7-e5eb6646c770',
  'src': 'CIVIL APPELLATE JURISDICTION',
  'text': 'CIVIL APPELLATE JURISDICTION',
  'tgt': 'दीवानी अपीलीय क्षेत्राधिकार'},
 {'id': '66ea0f20-8970-4ccf-bb53-5a7d74f8b285',
  's_id': 'fd77ae53-eadd-48a2-8555-3eb00db0583b',
  'src': 'Civil Appeal Nos 9367-9369 of 2011',
  'text': 'Civil Appeal Nos 9367-9369 of 2011',
  'tgt': '9367 की दीवानी अपील सं. 2011-9369'},
 {'id': 'd88ec932-f403-45d6-a290-1933a471ed27',
  's_id': 'd30d5348-1829-4ab4-ac3c-19ac102c0411',
  'src': 'The Secretary, Ministry of Defen

In [13]:
# replace the target token
for placeholder in placeholders:
    for tag in tags:
        if 'id' in tag.attrib:
            if tag.attrib['id'] == placeholder['id']:
                tag.text = placeholder['tgt']

In [8]:
save_document_xml(extracted_dir, document_xml)
save_docx(extracted_dir, filenames, os.path.join(output_dir, "kd2.docx"))